In [15]:
import numpy as np
import cv2
from glob import glob
import os




#動画からフレーム画像を抽出する関数
#一つの動画から任意の数のフレームを抽出する
def output_frame(num_frame):

    data_dir = './output'

    wake_video_dir_list = glob(os.path.join(data_dir, 'wake', 'video') + '/*.mp4')
    sleep_video_dir_list = glob(os.path.join(data_dir, 'sleep', 'video') + '/*.mp4')

    train_rate = 0.8
    valid_rate = 0.2

    num_data = len(wake_video_dir_list) #動画の数は起床と睡眠で同じ

    #データの分割
    train_num = int(num_data * train_rate)
    valid_num = int(train_num * valid_rate)
    train_num = train_num - valid_num
    test_num = num_data - train_num - valid_num

    wake_train = wake_video_dir_list[:train_num]
    wake_valid = wake_video_dir_list[train_num:train_num+valid_num]
    wake_test = wake_video_dir_list[train_num+valid_num:]

    sleep_train = sleep_video_dir_list[:train_num]
    sleep_valid = sleep_video_dir_list[train_num:train_num+valid_num]
    sleep_test = sleep_video_dir_list[train_num+valid_num:]

    #画像ファイルの出力先   
    train_frame_dir = os.path.join('train', 'images')
    valid_frame_dir = os.path.join('valid', 'images')
    test_frame_dir = os.path.join('test', 'images')

    print('フレーム画像の出力を開始します。')

    #動画ファイルをフレーム画像に変換

    #訓練データ
    video2frame(wake_train, train_frame_dir, num_frame, is_wake=True)
    video2frame(sleep_train, train_frame_dir, num_frame, is_wake=False)

    #検証データ
    video2frame(wake_valid, valid_frame_dir, num_frame, is_wake=True)
    video2frame(sleep_valid, valid_frame_dir, num_frame, is_wake=False)

    #テストデータ
    video2frame(wake_test, test_frame_dir, num_frame, is_wake=True)
    video2frame(sleep_test, test_frame_dir, num_frame, is_wake=False)


    print('フレーム画像の出力が完了しました。')


#動画からフレーム画像を等間隔に抽出する関数
def video2frame(video_dir_list, frame_dir, num_frame=10, is_wake=True):
    for video_dir in video_dir_list:
        cap = cv2.VideoCapture(video_dir)
        if not cap.isOpened():
            print('Error: Could not open video.')
            return

        frame_num = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        frame_rate = int(frame_num / num_frame)

        for i in range(num_frame):
            cap.set(cv2.CAP_PROP_POS_FRAMES, i * frame_rate)
            ret, frame = cap.read()
            if ret:
                if is_wake:
                    frame_name = 'wake_' + os.path.basename(video_dir) + '_frame' + '_wake_' + str(i) + '.jpg'
                else:
                    frame_name = 'sleep_' + os.path.basename(video_dir) + '_frame' + '_sleep_' + str(i) + '.jpg'
                frame_path = os.path.join(frame_dir, frame_name)
                cv2.imwrite(frame_path, frame)
            else:
                print('Error: Could not read frame.')
                break


        cap.release()

In [16]:
output_frame(50)

フレーム画像の出力を開始します。
フレーム画像の出力が完了しました。
